<a href="https://colab.research.google.com/github/shreeramshanmugasundaram/PytorchLearning/blob/main/simple_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Create a Model Class that inherits nn.
class Model(nn.Module):
    # Input Layer (4 Feature of Flower) --> Hidden Layer 1(number of Neurons) --> H2(n)--> output(3 class of Iris flower)
  def __init__(self, in_features = 4, h1 = 8, h2 = 9, out_features = 3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

In [3]:
# Pick a manula seed for randomization
torch.manual_seed(41)
#  Create an instance of model
model = Model()

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)

In [6]:
my_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [7]:
# change lsa column into a int
my_df['species'] = my_df['species'].replace('setosa', 0.0)
my_df['species'] = my_df['species'].replace('versicolor', 1.0)
my_df['species'] = my_df['species'].replace('virginica', 2.0)
my_df

<ipython-input-7-60a566bdc610>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  my_df['species'] = my_df['species'].replace('virginica', 2.0)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [8]:
# Train Test Split
X = my_df.drop('species',axis= 1)
y = my_df['species']

In [9]:
X = X.values
y = y.values

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 41)

In [12]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [13]:
# set the criterion of model to measure the error, how far off the prediction are from
criterion = nn.CrossEntropyLoss()
# Choose Adam Optimizer = lr = learning rate(if erro doesn't go down after a  bunch of iterations (echps), we wanna lower our learning rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [14]:
# Train our model
# Epochs ? (one run thru all the traning dagta in our netowrk)
epochs = 100
losses = []

for i in range(epochs):
  # Go forward and get a prediction =
  y_pred = model.forward(X_train)

  # Measure the loss /error, gonna be high at first
  loss = criterion(y_pred, y_train)

  # keep Track of our losses
  losses.append(loss.detach().numpy())

  # print every 10 epoch
  if i %10 ==0:
    print(f'Epoch: {i} and loss : {loss}')

  # Do some back propogation: take the error rate of forward propagation and feed it back
  # thru the network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epoch: 0 and loss : 1.1251550912857056
Epoch: 10 and loss : 1.0096259117126465
Epoch: 20 and loss : 0.8157405853271484
Epoch: 30 and loss : 0.585706353187561
Epoch: 40 and loss : 0.3999636471271515
Epoch: 50 and loss : 0.26768800616264343
Epoch: 60 and loss : 0.17942361533641815
Epoch: 70 and loss : 0.12151690572500229
Epoch: 80 and loss : 0.0860118493437767
Epoch: 90 and loss : 0.06520850956439972


In [ ]:
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

Text(0.5, 0, 'Epoch')

In [16]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)

In [17]:
loss

tensor(0.1315)

In [20]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    if y_test[i] == 0.0:
      print('Setosa')
    elif y_test[i] == 1.0:
      print('Versicolor')
    else:
      print('Virginica')

    print(f'{i+1}.) {str(y_val)}')
    print(f'Actual: {y_test[i]}')

    # if correct or not
    if y_val.argmax().item() == y_test[i]:
      correct += 1

  print(f'We got {correct} correct')

Virginica
1.) tensor([-5.8760,  4.4628,  6.5124])
Actual: 2
Virginica
2.) tensor([-7.5436,  4.1670,  9.7255])
Actual: 2
Virginica
3.) tensor([-8.4504,  5.1271, 10.1987])
Actual: 2
Versicolor
4.) tensor([-4.3398,  5.6267,  2.5608])
Actual: 1
Virginica
5.) tensor([-7.1825,  4.8754,  8.2992])
Actual: 2
Versicolor
6.) tensor([-3.3974,  5.3463,  1.2803])
Actual: 1
Virginica
7.) tensor([-5.9231,  4.9822,  5.9997])
Actual: 2
Versicolor
8.) tensor([-4.2881,  5.7003,  2.3889])
Actual: 1
Virginica
9.) tensor([-6.5356,  4.9257,  7.1260])
Actual: 2
Virginica
10.) tensor([-8.0509,  4.4129, 10.3286])
Actual: 2
Virginica
11.) tensor([-5.6762,  4.9500,  5.6215])
Actual: 2
Setosa
12.) tensor([ 4.5662, -2.2476, -2.8911])
Actual: 0
Setosa
13.) tensor([ 4.2571, -1.9967, -2.7330])
Actual: 0
Versicolor
14.) tensor([-2.1108,  4.0515,  0.5799])
Actual: 1
Setosa
15.) tensor([ 3.4575, -1.2098, -2.3514])
Actual: 0
Virginica
16.) tensor([-5.4724,  5.1165,  5.0934])
Actual: 2
Setosa
17.) tensor([ 4.0557, -1.7949, 

In [21]:
new_iris = torch.tensor([4.7,3.2,1.3,0.2])

In [23]:
with torch.no_grad():
  print(model(new_iris))
  # print(model.forward(new_iris))
  # print(model.forward(new_iris).argmax().item())

tensor([ 4.5365, -2.2383, -2.8684])


In [24]:
torch.save(model.state_dict(), 'iris_model.pt')

In [25]:
new_model = Model()
new_model.load_state_dict(torch.load('iris_model.pt'))

<ipython-input-25-7c1c09c835da>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model.load_state_dict(torch.load('iris_model.pt'))


<All keys matched successfully>

In [26]:
new_model.eval()

Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)